In [23]:
import numpy as np
import cv2
import os
import tensorflow as tf
from sklearn.utils import shuffle

In [24]:
train_path="C:\\Users\\asd\\Desktop\\Plant\\train\\Train\\"
#test_path="C:\\Users\\Yashvi Gulati\\Documents\\Pucho\\test"
imgsize=32
c=3

In [25]:
hot_array=np.zeros(12, dtype=int)

In [26]:
def label_function(img):
    file_name=img.split('.')[-3]
    hot_array=np.zeros(12, dtype=int)
    q=int(file_name[5]+file_name[6])-1
    hot_array[q]=1
    return hot_array

In [27]:
def createbatch(batchsize,j,array):
    batch=array[j:j+batchsize]
    imgarray=[]
    labelarray=[]
    for k in batch:
        filepath=os.path.join(train_path,k)
        name=label_function(os.path.basename(filepath))
        labelarray.append(name)
        img=cv2.imread(filepath)
        img= cv2.resize(img, (imgsize,imgsize))
        imgarray.append(img)
    
    train_data=np.array(imgarray)
    train_label=np.array(labelarray)
    
    return train_data, train_label

In [28]:
x = tf.placeholder(tf.float32, shape=[None, imgsize,imgsize,c])
y_= tf.placeholder(tf.float32, shape=[None,12])

In [29]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

In [30]:
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],strides=[1, 2, 2, 1], padding='SAME')

In [32]:
W_conv1 = weight_variable([5, 5, 3, 32])
b_conv1 = bias_variable([32])

h_conv1 = tf.nn.relu((conv2d(x, W_conv1) + b_conv1)) #20X128X128X32
h_pool1 = max_pool_2x2(h_conv1) #20X64X64X32

W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2) #20X64X64X64
h_pool2 = max_pool_2x2(h_conv2) #20X32X32X64

W_conv3 = weight_variable([5, 5, 32, 128])
b_conv3 = bias_variable([128])

h_conv3 = tf.nn.relu(conv2d(h_pool2, W_conv3) + b_conv3) #20X32X32X128
h_pool3 = max_pool_2x2(h_conv3) #20X16X16X128

W_fc1 = weight_variable([8 * 8 * 64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool3, [-1, 8*8*64]) #converting into 2d
h_fc1 = tf.nn.relu(tf.matmul(h_pool3_flat, W_fc1) + b_fc1)

keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

W_fc2 = weight_variable([1024, 12])
b_fc2 = bias_variable([12])

y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2

In [33]:
cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv))

In [37]:
train_step = tf.train.AdamOptimizer(1e-2).minimize(cross_entropy)

In [38]:
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [40]:
with tf.Session() as sess:#declaring tf.Session as sess
    sess.run(tf.global_variables_initializer())#initializing all the variables we used
    for j in range(10) :#epochs 
        imList=shuffle(os.listdir(train_path))
        for i in range(24):#number of iterations we are using
            batchData,batchLabel = createbatch(190,i*190,imList)
            train_step.run(feed_dict={x: batchData, y_: batchLabel, keep_prob: 0.7})#dropout probabilty during training is kept 0.5
        train_accuracy = 0
        valid_accuracy = 0
        for i in range(24):#number of iterations we are using
            batchData,batchLabel = createbatch(190,i*190,imList)
            train_accuracy += accuracy.eval(feed_dict={x: batchData, y_: batchLabel, keep_prob: 1})#training accuracy calculated with no dropout probabilty
        print('epoch %d, training accuracy %g' % (j, train_accuracy / 24))
        valData,valLabel=createbatch(190,24*190,imList)
        valid_accuracy += accuracy.eval(feed_dict={x: valData, y_: valLabel, keep_prob: 1})

epoch 0, training accuracy 0.133895
epoch 1, training accuracy 0.159368
epoch 2, training accuracy 0.175158
epoch 3, training accuracy 0.199158
epoch 4, training accuracy 0.227158
epoch 5, training accuracy 0.251158
epoch 6, training accuracy 0.261684
epoch 7, training accuracy 0.280842
epoch 8, training accuracy 0.286737
epoch 9, training accuracy 0.277895


In [45]:
print('Final Training Accuracy is', train_accuracy*(100/24), '%')
print('Final Training Accuracy is', valid_accuracy*100, '%')

Final Training Accuracy is 28.9473685126 %
Final Training Accuracy is 28.9473682642 %
